In [1]:
import findspark
findspark.init("C:/spark") 

from pyspark import SparkContext
sc= SparkContext("local","pyspark week6")

In [2]:
from pyspark.sql import SparkSession
spark= SparkSession.builder.getOrCreate()

# Lab6

In [3]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [4]:
columns= [
    "Elevation", "Aspect", "Slope","Horizontal_Distance_To_Hydrology", "Vertical_Distance_To_Hydrology",
    "Horizontal_Distance_To_Roadways", "Hillshade_9am", "Hillshade_Noon", "Hillshade_3pm"," Horizontal_Distance_To_Fire_Points",
    *[f"Wilderness_Area{i+1}" for i in range(4)],
    *[f"Soil_Type{i+1}" for i in range(40)],
    "Cover_Type"
]
print(len(columns))

55


In [5]:
data_no_head= spark.read.option("InferSchema",True).option('header',False).csv('covtype.data')
data_no_head.show(2)

+----+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
| _c0|_c1|_c2|_c3|_c4|_c5|_c6|_c7|_c8| _c9|_c10|_c11|_c12|_c13|_c14|_c15|_c16|_c17|_c18|_c19|_c20|_c21|_c22|_c23|_c24|_c25|_c26|_c27|_c28|_c29|_c30|_c31|_c32|_c33|_c34|_c35|_c36|_c37|_c38|_c39|_c40|_c41|_c42|_c43|_c44|_c45|_c46|_c47|_c48|_c49|_c50|_c51|_c52|_c53|_c54|
+----+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|2596| 51|  3|258|  0|510|221|232|148|6279|   1|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|

In [6]:
df1= data_no_head.toDF(*columns).withColumn('Cover_Type', col('Cover_Type').cast(DoubleType()))
df1.show(2)

+---------+------+-----+--------------------------------+------------------------------+-------------------------------+-------------+--------------+-------------+-----------------------------------+----------------+----------------+----------------+----------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+----------+
|Elevation|Aspect|Slope|Horizontal_Distance_To_Hydrology|Vertical_Distance_To_Hydrology|Horizontal_Distance_To_Roadways|Hillshade_9am|Hillshade_Noon|Hillshade_3pm| Horizontal_Distance_To_Fire_Points|Wilderness_Area1|Wilderness_Area2|Wilderness_Area3|

In [7]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [9]:
vector_cols= columns[:-1]


In [12]:
dec_assembler= VectorAssembler(inputCols=vector_cols, outputCol='features')

(dec_test,dec_train)= df1.randomSplit([0.9,0.1])
dec_train.cache()
dec_test.cache()

DataFrame[Elevation: int, Aspect: int, Slope: int, Horizontal_Distance_To_Hydrology: int, Vertical_Distance_To_Hydrology: int, Horizontal_Distance_To_Roadways: int, Hillshade_9am: int, Hillshade_Noon: int, Hillshade_3pm: int,  Horizontal_Distance_To_Fire_Points: int, Wilderness_Area1: int, Wilderness_Area2: int, Wilderness_Area3: int, Wilderness_Area4: int, Soil_Type1: int, Soil_Type2: int, Soil_Type3: int, Soil_Type4: int, Soil_Type5: int, Soil_Type6: int, Soil_Type7: int, Soil_Type8: int, Soil_Type9: int, Soil_Type10: int, Soil_Type11: int, Soil_Type12: int, Soil_Type13: int, Soil_Type14: int, Soil_Type15: int, Soil_Type16: int, Soil_Type17: int, Soil_Type18: int, Soil_Type19: int, Soil_Type20: int, Soil_Type21: int, Soil_Type22: int, Soil_Type23: int, Soil_Type24: int, Soil_Type25: int, Soil_Type26: int, Soil_Type27: int, Soil_Type28: int, Soil_Type29: int, Soil_Type30: int, Soil_Type31: int, Soil_Type32: int, Soil_Type33: int, Soil_Type34: int, Soil_Type35: int, Soil_Type36: int, S

In [13]:
fin_out= dec_assembler.transform(dec_train)
fin_out.show(2)

+---------+------+-----+--------------------------------+------------------------------+-------------------------------+-------------+--------------+-------------+-----------------------------------+----------------+----------------+----------------+----------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+----------+--------------------+
|Elevation|Aspect|Slope|Horizontal_Distance_To_Hydrology|Vertical_Distance_To_Hydrology|Horizontal_Distance_To_Roadways|Hillshade_9am|Hillshade_Noon|Hillshade_3pm| Horizontal_Distance_To_Fire_Points|Wilderness_Area1|Wilderness_Ar

In [15]:
tree= DecisionTreeClassifier(labelCol='Cover_Type', featuresCol='features', maxDepth=15)

dec_model= tree.fit(fin_out)

In [16]:
dec_preds= dec_model.transform(fin_out)

dec_preds.show(2)

+---------+------+-----+--------------------------------+------------------------------+-------------------------------+-------------+--------------+-------------+-----------------------------------+----------------+----------------+----------------+----------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+----------+--------------------+--------------------+--------------------+----------+
|Elevation|Aspect|Slope|Horizontal_Distance_To_Hydrology|Vertical_Distance_To_Hydrology|Horizontal_Distance_To_Roadways|Hillshade_9am|Hillshade_Noon|Hillshade_3pm| Horizontal_D

In [17]:
dec_conf= dec_preds.groupBy('Cover_Type').pivot('prediction').count().fillna(0)
dec_conf.show()

+----------+-----+-----+----+---+---+----+----+
|Cover_Type|  1.0|  2.0| 3.0|4.0|5.0| 6.0| 7.0|
+----------+-----+-----+----+---+---+----+----+
|       7.0|  211|    9|   0|  0|  0|   0|1898|
|       1.0|17999| 3240|   0|  0| 10|   1|  38|
|       4.0|    0|    0|  28|228|  0|  24|   0|
|       3.0|    0|  250|3020|  7|  0| 276|   0|
|       2.0| 1801|26306|  30|  1| 48| 137|   0|
|       6.0|    0|  306| 144|  0|  1|1323|   0|
|       5.0|    5|  464|   1|  0|412|   4|   0|
+----------+-----+-----+----+---+---+----+----+



In [19]:
dec_evl= MulticlassClassificationEvaluator(
    labelCol='Cover_Type', predictionCol='prediction', metricName='accuracy'
)
acc_dec1= dec_evl.evaluate(dec_preds)
print(f"accuracy on train: {acc_dec1*100:.4f}%")

accuracy on train: 87.9152%


In [21]:
fin_test_out= dec_assembler.transform(dec_test)
dec_preds_test= dec_model.transform(fin_test_out)
acc_dec2= dec_evl.evaluate(dec_preds_test)

In [23]:
print(f"accuracy on train: {acc_dec2*100:.4f}%")
conf_dec2= dec_preds_test.groupBy('Cover_Type').pivot('prediction').count().fillna(0)
conf_dec2.show()

accuracy on train: 81.2552%
+----------+------+------+-----+----+----+----+-----+
|Cover_Type|   1.0|   2.0|  3.0| 4.0| 5.0| 6.0|  7.0|
+----------+------+------+-----+----+----+----+-----+
|       7.0|  3410|   445|    0|   0|   0|   0|14537|
|       1.0|148099| 40002|   28|   0| 188|  43| 2192|
|       4.0|     0|    21|  664|1510|   0| 272|    0|
|       3.0|    71|  3090|24474| 415|  24|4127|    0|
|       2.0| 27311|223693|  937|  58|1026|1749|  204|
|       6.0|    69|  3375| 2721| 130|  15|9283|    0|
|       5.0|   145|  5055|  156|   0|3198|  53|    0|
+----------+------+------+-----+----+----+----+-----+



In [24]:
from pyspark.ml import Pipeline
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit

In [35]:
dtree= DecisionTreeClassifier(
    labelCol= 'Cover_Type', featuresCol='features'
)

In [44]:
pipe_dec= Pipeline(stages=[dec_assembler,dtree])

pgrid_dec= ParamGridBuilder().addGrid(dtree.maxDepth,[10,15,20])\
            .addGrid(dtree.minInstancesPerNode, [1,4,5]).\
            addGrid(dtree.impurity, ['gini']).build()

dec_evaluator= MulticlassClassificationEvaluator(
    labelCol= 'Cover_Type', predictionCol='prediction',metricName= 'accuracy'
)

In [45]:
dec_tvs= TrainValidationSplit(
    estimator= pipe_dec, estimatorParamMaps= pgrid_dec,evaluator= dec_evaluator,
    trainRatio= 0.8
)

In [46]:
tvs_model= dec_tvs.fit(dec_train)

In [49]:
preds_dec_pipe= tvs_model.transform(dec_test)

acc_dec_pipe= dec_evaluator.evaluate(preds_dec_pipe)
print(f'tuning accuracy: {acc_dec_pipe}')

tuning accuracy: 0.8336846534937546


In [48]:
best_dec_model= tvs_model.bestModel.stages[-1]
best_dec_model.getOrDefault('impurity')

'gini'